In [2]:
import csv
import pandas as pd
from paper import Paper, PaperGallery

The following block is an example block to show how to read a csv and preview first several lines.

In [3]:
main_dataset = pd.read_csv("IEEE VIS papers 1990-2020 - Main dataset.csv")
main_dataset.head()


,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount,XploreCitationCount,PubsCited,Award
0,InfoVis,2020,A Bayesian cognition approach for belief updat...,10.1109/TVCG.2020.3029412,http://dx.doi.org/10.1109/TVCG.2020.3029412,978,988,J,Understanding correlation judgement is importa...,Alireza Karduni;Douglas Markant;Ryan Wesslen;W...,Alireza Karduni;Douglas Markant;Ryan Wesslen;W...,"University of North Carolina, Charlotte;Univer...","10.1109/TVCG.2014.2346979,10.1109/TVCG.2019.29...","Information visualization,Bayesian modeling,un...",NaN,0.0,60.0,NaN
1,InfoVis,2020,A Design Space of Vision Science Methods for V...,10.1109/TVCG.2020.3029413,http://dx.doi.org/10.1109/TVCG.2020.3029413,1117,1127,J,A growing number of efforts aim to understand ...,Madison A. Elliott;Christine Nothelfer;Cindy X...,Madison A. Elliott;Christine Nothelfer;Cindy X...,The University of British Columbia;Northwester...,"10.1109/TVCG.2015.2467732,10.1109/INFVIS.1997....","Perception,human vision,empirical research,eva...",NaN,0.0,83.0,HM
2,InfoVis,2020,A Generic Framework and Library for Exploratio...,10.1109/TVCG.2020.3028948,http://dx.doi.org/10.1109/TVCG.2020.3028948,358,368,J,Small multiples are miniature representations ...,Fritz Lekschas;Xinyi Zhou;Wei Chen 0001;Nils G...,Fritz Lekschas;Xinyi Zhou;Wei Chen;Nils Gehlen...,"Cambridge, Harvard School of Engineering and A...","10.1109/TVCG.2015.2467851,10.1109/VISUAL.2005....","Information visualization,small multiples,inte...",NaN,0.0,52.0,HM
3,InfoVis,2020,A Simple Pipeline for Coherent Grid Maps,10.1109/TVCG.2020.3028953,http://dx.doi.org/10.1109/TVCG.2020.3028953,1236,1246,J,Grid maps are spatial arrangements of simple t...,Wouter Meulemans;Max Sondag;Bettina Speckmann,Wouter Meulemans;Max Sondag;Bettina Speckmann,TU Eindhoven;TU Eindhoven;TU Eindhoven,"10.1109/TVCG.2008.165,10.1109/TVCG.2011.174,10...","Grid maps,algorithms,tile maps,small multiples...",NaN,0.0,52.0,HM
4,InfoVis,2020,A Structured Review of Data Management Technol...,10.1109/TVCG.2020.3028891,http://dx.doi.org/10.1109/TVCG.2020.3028891,1128,1138,J,"In the last two decades, interactive visualiza...",Leilani Battle;Carlos Scheidegger,Leilani Battle;Carlos Scheidegger,Department of Computer ScienceUniversity of Ma...,"10.1109/INFVIS.2005.1532136,10.1109/TVCG.2008....",NaN,NaN,0.0,194.0,NaN


In [7]:
gallery = PaperGallery()
gallery.construct_gallery_from_main_dataset()
gallery.gallery

NameError: name 'PaperGallery' is not defined

In [6]:
len(main_dataset)

3394